In [1]:
import DualwormFunctions as dw
import Observables as obs
%load_ext line_profiler
%load_ext memory_profiler
import RunBasis_NewSaves
import argparse
import StartStates as strst
import numpy as np

In [2]:
nst = 2;
nsm = 2;
nips = 2;
nrps = 4;
nb = 2;
measperiod = 2;
nt = 64
nh = 64

In [3]:
def genArgs():
    parser = argparse.ArgumentParser()
    
    parser.add_argument('-f', type = str)

    parser.add_argument('--L', type = int, default = 12, help = 'Lattice side size')

    # COUPLINGS
    parser.add_argument('--J1', type = float, default = 1,
                        help = 'NN coupling') # nearest-neighbour coupling
    parser.add_argument('--J2', type = float, default = 0,
                        help = '2nd NN coupling') # 2nd NN coupling
    parser.add_argument('--J3', type = float, default = 0,
                        help = '3rd NN coupling') # 3rd NN coupling
    parser.add_argument('--J4', type = float, default = 0.0,
                        help = '4th NN coupling')
    parser.add_argument('--h', type = float, default = 1.0,
                        help = 'Magnetic field')    
    #NUMBER OF STEPS AND ITERATIONS
    parser.add_argument('--nst', type = int, default = nst,
                        help = 'number of thermalisation steps') # number of thermalisation steps
    parser.add_argument('--nsm', type = int, default = nsm,
                        help = 'number of measurements steps') # number of measurement steps
    parser.add_argument('--nips', type = int, default = nips,
                        help = 'number of worm constructions per MC step')
    parser.add_argument('--nrps', type = int, default = nrps,
                        help = 'number of replica loops per MC step')
    parser.add_argument('--measperiod', type = int, default = measperiod,
                        help = 'number of nips worm building + swaps between measurements')
    parser.add_argument('--nb', type = int, default = nb,
                        help = 'number of bins')

    #PARALLELISATION
    parser.add_argument('--ncores', type = int, default = 6,
                        help = 'number of threads to use')

    
    # INITIALISATION PARAMETERS
    parser.add_argument('--randominit', default = True, action ='store_true',
                        help = 'intialise the states randomly')
    parser.add_argument('--same', default = False, action = 'store_true',
                        help = '''initialise all temperatures with the same
                        state (debug purposes)''')
    parser.add_argument('--magninit', default = False, action = 'store_true',
                        help = '''initialise all the temperature with
                        one of the m=1/3 GS''')
    parser.add_argument('--magnstripes', default = False, action = 'store_true',
                       help = '''initialise all the temperature with
                       m=1/3 stripes''')
    parser.add_argument('--maxflip', default = False, action = 'store_true',
                       help = '''initialise all the temperature with
                       maximally flippable plateau''')
    parser.add_argument('--loadfromfile', default = False, action = 'store_true',
                       help = '''initialise all the states with
                       results from a previously performed simulations''')
    parser.add_argument('--filename', type = str, default = 'tests/J1hLongRun_RndInit_AlternateUpdateResults',
                       help = '''initialise all the states with
                       results from a previously performed simulations''')
    
    #WORM PARAMETERS
    parser.add_argument('--nmaxiter', type = int, default = 10,
                        help = '''maximal number of segments in a loop update over the
                        size of the lattice (1 = 1times the number of dualbonds in the
                        lattice)''')
    parser.add_argument('--measupdate', default = False, action = 'store_true',
                       help = '''activate to mimic the action of the measuring tip''')
    parser.add_argument('--p', type = float, default = 0.0, 
                       help = '''prob of the measuring tip flipping the spin (number between 0 and 1)''')
    parser.add_argument('--ssf', default = True, action = 'store_true',
                        help = 'activate for single spin flip update')
    parser.add_argument('--alternate', default = True, action = 'store_true',
                        help = 'activate for single spin flip update and dw update')
    parser.add_argument('--checkgs', default = True, action = 'store_true',
                        help = 'activate to debug ssf')
    
    #TEMPERATURE PARAMETERS
    parser.add_argument('--t_list', nargs = '+', type = float, default = [0.05,60],
                        help = 'list of limiting temperature values')
    parser.add_argument('--nt_list', nargs = '+', type = int, default = [nt],
                        help = 'list of number of temperatures in between the given limiting temperatures')
    parser.add_argument('--log_tlist', default = False, action='store_true',
                        help = 'state whether you want the temperature be spaced log-like or linear-like (activate if you want log)')
    parser.add_argument('--stat_temps_lims', nargs = '+', type = float,
                        help = '''limiting temperatures for the various ranges of
                        measurements''') 
                        #default will be set to none, and then we can decide what to do later on.
    #MAGNETIC FIELD PARAMETERS
    parser.add_argument('--h_list', nargs = '+', default = [0,6.0],type = float,
                        help = 'list of limiting magnetic field values')
    parser.add_argument('--nh_list', nargs = '+', default = [nh], type = int,
                        help = 'list of number of magnetic fields in between the given limiting temperatures')
    parser.add_argument('--stat_hfields_lims', nargs = '+', type = float,
                help = '''limiting magnetic fields for the various ranges of
                measurements''') 
    #CORRELATIONS PARAMETER
    parser.add_argument('--energy', default = True, action = 'store_true',
                        help = 'activate if you want to save the energy')
    parser.add_argument('--magnetisation', default =True, action = 'store_true',
                        help = 'activate if you want to save the magnetisation')
    parser.add_argument('--charges', default = False, action = 'store_true',
                        help = 'activate if you want to save the charges')
    parser.add_argument('--correlations', default = True, action = 'store_true',
                        help = 'activate if you want to save either central or all correlations')
    parser.add_argument('--all_correlations', default = False, action = 'store_true',
                        help = '''activate if you want to save the correlations for all non-equivalent
                        pairs of sites. Otherwise, will save central correlations.''')
    parser.add_argument('--firstcorrelations', default = False, action = 'store_true',
                        help = 'activate if you want to save first correlations, otherwise will save central')
    #SAVE
    parser.add_argument('--output', type = str, default = "tests/test", help = 'saving filename (.pkl will be added)')
    args = parser.parse_args()
    return args

In [4]:
def callRunBasis():
    args = genArgs()
    return RunBasis_NewSaves.main(args)

In [5]:
%lprun -f dw.mcs_swaps meanstat, swapst, swapsh, failedupdatesth, failedupdates = callRunBasis() 

-------------------Initialisation--------------------
Lattice side size:  12
J1  1
J2  0
J3  0
J3st  0
Couplings extracted
Hamiltonian expression (without field) computed
single spin flip update
alternating ssf and dw update
Number of temperatures:  64
Temperatures: [5.00000000e-02 1.00158730e+00 1.95317460e+00 2.90476190e+00
 3.85634921e+00 4.80793651e+00 5.75952381e+00 6.71111111e+00
 7.66269841e+00 8.61428571e+00 9.56587302e+00 1.05174603e+01
 1.14690476e+01 1.24206349e+01 1.33722222e+01 1.43238095e+01
 1.52753968e+01 1.62269841e+01 1.71785714e+01 1.81301587e+01
 1.90817460e+01 2.00333333e+01 2.09849206e+01 2.19365079e+01
 2.28880952e+01 2.38396825e+01 2.47912698e+01 2.57428571e+01
 2.66944444e+01 2.76460317e+01 2.85976190e+01 2.95492063e+01
 3.05007937e+01 3.14523810e+01 3.24039683e+01 3.33555556e+01
 3.43071429e+01 3.52587302e+01 3.62103175e+01 3.71619048e+01
 3.81134921e+01 3.90650794e+01 4.00166667e+01 4.09682540e+01
 4.19198413e+01 4.28714286e+01 4.38230159e+01 4.47746032e+01
 

List of measurements to be performed: ['Energy', 'Magnetisation', 'Si', 'Central_Correlations']
List of field indices to measure: range(0, 64)
-----------Thermalisation------------------
Number of thermalisation steps =  2
itermcs =  2
iterreplicas =  4
ssf =  True
alternate =  True
statsfunctions []
Time for mcsevolve = 3.725808620452881
Time for tempering = 0.1791515350341797
Time for mapping to spins + computing statistics= 2.4318695068359375e-05
Time for all thermalisation steps =  7.811364650726318
RunBasis: Issue at temperature index  0  and h index  2
   energies[t,h] =  -920.3809523809547
   H0[t,h] =  -864.0
   magntot[t,h]  296
   new_E[t,h] = H0[t,h] - h*magntot[t,h] -920.3809523809524
RunBasis: Issue at temperature index  0  and h index  3
   energies[t,h] =  -949.7142857142868
   H0[t,h] =  -864.0
   magntot[t,h]  300
   new_E[t,h] = H0[t,h] - h*magntot[t,h] -949.7142857142857
RunBasis: Issue at temperature index  0  and h index  5
   energies[t,h] =  -1001.1428571428547
 

   energies[t,h] =  -1634.857142857149
   H0[t,h] =  -8.0
   magntot[t,h]  438
   new_E[t,h] = H0[t,h] - h*magntot[t,h] -1634.8571428571427
RunBasis: Issue at temperature index  6  and h index  40
   energies[t,h] =  -1758.2857142857067
   H0[t,h] =  116.0
   magntot[t,h]  492
   new_E[t,h] = H0[t,h] - h*magntot[t,h] -1758.2857142857142
RunBasis: Issue at temperature index  6  and h index  41
   energies[t,h] =  -1848.5714285714384
   H0[t,h] =  96.0
   magntot[t,h]  498
   new_E[t,h] = H0[t,h] - h*magntot[t,h] -1848.5714285714284
RunBasis: Issue at temperature index  6  and h index  42
   energies[t,h] =  -1820.0000000000048
   H0[t,h] =  140.0
   magntot[t,h]  490
   new_E[t,h] = H0[t,h] - h*magntot[t,h] -1820.0
RunBasis: Issue at temperature index  6  and h index  43
   energies[t,h] =  -1929.904761904775
   H0[t,h] =  216.0
   magntot[t,h]  524
   new_E[t,h] = H0[t,h] - h*magntot[t,h] -1929.904761904762
RunBasis: Issue at temperature index  6  and h index  44
   energies[t,h] =  -2

   H0[t,h] =  28.0
   magntot[t,h]  338
   new_E[t,h] = H0[t,h] - h*magntot[t,h] -1549.333333333333
RunBasis: Issue at temperature index  15  and h index  50
   energies[t,h] =  -1625.33333333334
   H0[t,h] =  108.0
   magntot[t,h]  364
   new_E[t,h] = H0[t,h] - h*magntot[t,h] -1625.3333333333333
RunBasis: Issue at temperature index  15  and h index  51
   energies[t,h] =  -1562.8571428571365
   H0[t,h] =  40.0
   magntot[t,h]  330
   new_E[t,h] = H0[t,h] - h*magntot[t,h] -1562.8571428571427
RunBasis: Issue at temperature index  15  and h index  52
   energies[t,h] =  -1826.8571428571406
   H0[t,h] =  -44.0
   magntot[t,h]  360
   new_E[t,h] = H0[t,h] - h*magntot[t,h] -1826.857142857143
RunBasis: Issue at temperature index  15  and h index  53
   energies[t,h] =  -1650.8571428571488
   H0[t,h] =  -76.0
   magntot[t,h]  312
   new_E[t,h] = H0[t,h] - h*magntot[t,h] -1650.857142857143
RunBasis: Issue at temperature index  15  and h index  54
   energies[t,h] =  -1706.8571428571458
   H0[t

   H0[t,h] =  -48.0
   magntot[t,h]  112
   new_E[t,h] = H0[t,h] - h*magntot[t,h] -613.3333333333333
RunBasis: Issue at temperature index  33  and h index  55
   energies[t,h] =  -570.6666666666633
   H0[t,h] =  16.0
   magntot[t,h]  112
   new_E[t,h] = H0[t,h] - h*magntot[t,h] -570.6666666666666
RunBasis: Issue at temperature index  33  and h index  56
   energies[t,h] =  -941.3333333333345
   H0[t,h] =  72.0
   magntot[t,h]  190
   new_E[t,h] = H0[t,h] - h*magntot[t,h] -941.3333333333333
RunBasis: Issue at temperature index  33  and h index  58
   energies[t,h] =  -1089.714285714287
   H0[t,h] =  4.0
   magntot[t,h]  198
   new_E[t,h] = H0[t,h] - h*magntot[t,h] -1089.7142857142858
RunBasis: Issue at temperature index  33  and h index  60
   energies[t,h] =  -1311.999999999998
   H0[t,h] =  48.0
   magntot[t,h]  238
   new_E[t,h] = H0[t,h] - h*magntot[t,h] -1311.9999999999998
RunBasis: Issue at temperature index  33  and h index  63
   energies[t,h] =  -1315.9999999999989
   H0[t,h] =

...

   new_E[t,h] = H0[t,h] - h*magntot[t,h] -1328.0
RunBasis: Issue at temperature index  6  and h index  36
   energies[t,h] =  -1428.000000000002
   H0[t,h] =  -36.0
   magntot[t,h]  406
   new_E[t,h] = H0[t,h] - h*magntot[t,h] -1428.0
RunBasis: Issue at temperature index  6  and h index  37
   energies[t,h] =  -1571.047619047621
   H0[t,h] =  64.0
   magntot[t,h]  464
   new_E[t,h] = H0[t,h] - h*magntot[t,h] -1571.047619047619
RunBasis: Issue at temperature index  6  and h index  38
   energies[t,h] =  -1544.1904761904739
   H0[t,h] =  12.0
   magntot[t,h]  430
   new_E[t,h] = H0[t,h] - h*magntot[t,h] -1544.190476190476
RunBasis: Issue at temperature index  6  and h index  40
   energies[t,h] =  -1807.4285714285738
   H0[t,h] =  44.0
   magntot[t,h]  486
   new_E[t,h] = H0[t,h] - h*magntot[t,h] -1807.4285714285713
RunBasis: Issue at temperature index  6  and h index  41
   energies[t,h] =  -1804.7619047619105
   H0[t,h] =  132.0
   magntot[t,h]  496
   new_E[t,h] = H0[t,h] - h*magntot[

RunBasis: Issue at temperature index  15  and h index  42
   energies[t,h] =  -1264.0000000000023
   H0[t,h] =  -152.0
   magntot[t,h]  278
   new_E[t,h] = H0[t,h] - h*magntot[t,h] -1264.0
RunBasis: Issue at temperature index  15  and h index  43
   energies[t,h] =  -1180.7619047619023
   H0[t,h] =  56.0
   magntot[t,h]  302
   new_E[t,h] = H0[t,h] - h*magntot[t,h] -1180.7619047619046
RunBasis: Issue at temperature index  15  and h index  46
   energies[t,h] =  -1240.3809523809487
   H0[t,h] =  -40.0
   magntot[t,h]  274
   new_E[t,h] = H0[t,h] - h*magntot[t,h] -1240.3809523809523
RunBasis: Issue at temperature index  15  and h index  48
   energies[t,h] =  -1399.9999999999948
   H0[t,h] =  8.0
   magntot[t,h]  308
   new_E[t,h] = H0[t,h] - h*magntot[t,h] -1400.0
RunBasis: Issue at temperature index  15  and h index  50
   energies[t,h] =  -1433.9047619047635
   H0[t,h] =  -72.0
   magntot[t,h]  286
   new_E[t,h] = H0[t,h] - h*magntot[t,h] -1433.904761904762
RunBasis: Issue at temperat

RunBasis: Issue at temperature index  32  and h index  50
   energies[t,h] =  -701.5238095238083
   H0[t,h] =  -92.0
   magntot[t,h]  128
   new_E[t,h] = H0[t,h] - h*magntot[t,h] -701.5238095238095
RunBasis: Issue at temperature index  32  and h index  55
   energies[t,h] =  -1193.3333333333314
   H0[t,h] =  -20.0
   magntot[t,h]  224
   new_E[t,h] = H0[t,h] - h*magntot[t,h] -1193.3333333333333
RunBasis: Issue at temperature index  32  and h index  56
   energies[t,h] =  -993.3333333333348
   H0[t,h] =  -12.0
   magntot[t,h]  184
   new_E[t,h] = H0[t,h] - h*magntot[t,h] -993.3333333333333
RunBasis: Issue at temperature index  32  and h index  59
   energies[t,h] =  -935.0476190476202
   H0[t,h] =  -36.0
   magntot[t,h]  160
   new_E[t,h] = H0[t,h] - h*magntot[t,h] -935.047619047619
RunBasis: Issue at temperature index  33  and h index  40
   energies[t,h] =  -499.42857142857247
   H0[t,h] =  -88.0
   magntot[t,h]  108
   new_E[t,h] = H0[t,h] - h*magntot[t,h] -499.4285714285714
RunBasis

/home/jcolbois/.local/lib/python3.6/site-packages/hickle/hickle.py:403: SerializedWarning: <class 'safe.safe'> type not understood, data have been serialized
  SerializedWarning)


Job done


In [6]:
print(failedupdatesth/(nst*nips))

[[0.54031636 0.94675926 0.92091049 ... 0.87866512 0.88175154 0.88213735]
 [0.5464892  0.57523148 0.61516204 ... 0.85725309 0.8599537  0.86496914]
 [0.45910494 0.46219136 0.50405093 ... 0.76793981 0.77527006 0.78337191]
 ...
 [0.02835648 0.02893519 0.02854938 ... 0.09375    0.09664352 0.0929784 ]
 [0.02546296 0.02797068 0.02739198 ... 0.09182099 0.09895833 0.0943287 ]
 [0.02102623 0.02372685 0.02411265 ... 0.08641975 0.0929784  0.09182099]]


In [7]:
print(failedupdates/(nsm*measperiod*nips))

[[0.55131173 0.98061343 0.99807099 ... 0.99855324 0.99932485 0.99893904]
 [0.54388503 0.57908951 0.61824846 ... 0.95225694 0.96489198 0.96865355]
 [0.45283565 0.48099923 0.49710648 ... 0.81404321 0.82214506 0.82677469]
 ...
 [0.02237654 0.02642747 0.02411265 ... 0.09857253 0.09702932 0.10407022]
 [0.02584877 0.02575231 0.02912809 ... 0.09124228 0.09866898 0.10108025]
 [0.02469136 0.02633102 0.02546296 ... 0.09673997 0.09722222 0.09481096]]


In [8]:
print(4*swapst/(nrps*nsm*measperiod*nh))

[0.         0.         0.         0.03125    0.08203125 0.16796875
 0.328125   0.43359375 0.5        0.58984375 0.58203125 0.65234375
 0.74609375 0.703125   0.7578125  0.79296875 0.7890625  0.83984375
 0.83203125 0.87109375 0.875      0.86328125 0.90234375 0.9140625
 0.87890625 0.91796875 0.8671875  0.91796875 0.92578125 0.91796875
 0.92578125 0.94140625 0.95703125 0.953125   0.96484375 0.94140625
 0.9296875  0.9453125  0.96875    0.95703125 0.94921875 0.94140625
 0.95703125 0.94921875 0.97265625 0.96484375 0.98046875 0.96875
 0.99609375 0.9609375  0.9921875  0.97265625 0.9765625  0.9609375
 0.9765625  0.97265625 0.98828125 0.97265625 0.9765625  0.98828125
 0.9921875  0.9921875  0.9765625  0.        ]


In [9]:
print(4*swapsh/(nrps*nsm*measperiod*nt))

[0.4296875  1.35546875 0.46875    1.328125   0.4609375  1.359375
 0.4140625  1.35546875 0.44921875 1.35546875 0.4296875  1.375
 0.46875    1.34375    0.4296875  1.33203125 0.45703125 1.34375
 0.4453125  1.3359375  0.453125   1.3046875  0.421875   1.31640625
 0.45703125 1.34375    0.4375     1.37109375 0.45703125 1.32421875
 0.484375   1.3515625  0.45703125 1.3828125  0.453125   1.34375
 0.44140625 1.32421875 0.4453125  1.34765625 0.46875    1.32421875
 0.4609375  1.33203125 0.45703125 1.36328125 0.453125   1.33984375
 0.44140625 1.3515625  0.4453125  1.37890625 0.46484375 1.34765625
 0.43359375 1.33203125 0.44921875 1.37890625 0.4609375  1.29296875
 0.453125   1.3359375  0.44921875 0.47265625]
